<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in Toronto</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


Download all the dependencies that we will need.


In [24]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import collections

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>


## 1. Download and Explore Dataset


We only manage to find some neighborhoods in Toronto for the project.


In [25]:
NeighbourhoodInDistrict = collections.namedtuple('NeighbourhoodInDistrict', ['district', 'neighbourhood'])
neighbourhood_tuples = []

filename = "toronto_neighbourhoods_list.txt"
with open(filename, 'r') as f:
    district = "INVALID"
    for line in f:
        if line[:2] == '--':
            district = line[2:-1] # exclude '\n'
            continue
        neighbourhood = line.split(',')[0]
        if neighbourhood[-1] == '\n':
            neighbourhood = neighbourhood[:-1]
        print('D: {}, N: {}'.format(district, neighbourhood))
        neighbourhood_tuples.append(NeighbourhoodInDistrict(district, neighbourhood))

D: Old Toronto, N: Alexandra Park
D: Old Toronto, N: The Annex
D: Old Toronto, N: Baldwin Village
D: Old Toronto, N: Cabbagetown
D: Old Toronto, N: CityPlace
D: Old Toronto, N: Chinatown
D: Old Toronto, N: Church and Wellesley
D: Old Toronto, N: Corktown
D: Old Toronto, N: Discovery District
D: Old Toronto, N: Distillery District
D: Old Toronto, N: Entertainment District
D: Old Toronto, N: East Bayfront
D: Old Toronto, N: Fashion District
D: Old Toronto, N: Financial District
D: Old Toronto, N: Garden District
D: Old Toronto, N: Grange Park
D: Old Toronto, N: Harbord Village
D: Old Toronto, N: Harbourfront
D: Old Toronto, N: Kensington Market
D: Old Toronto, N: First Chinatown
D: Old Toronto, N: Moss Park
D: Old Toronto, N: Old Town
D: Old Toronto, N: Quayside
D: Old Toronto, N: Queen Street West
D: Old Toronto, N: Regent Park
D: Old Toronto, N: South Core
D: Old Toronto, N: St. James Town
D: Old Toronto, N: St. Lawrence
D: Old Toronto, N: Toronto Islands
D: Old Toronto, N: Trefann Cou

#### Tranform the data into a _pandas_ dataframe


The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.


In [34]:
geolocator = Nominatim(user_agent="toronto_explorer")
# define the dataframe columns
column_names = ['District', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for nt in neighbourhood_tuples:
    address = '{}, {}, Toronto, Ontario'.format(nt.neighbourhood, nt.district)
    location = geolocator.geocode(address)
    if not location:
        print('cannot find latitude and longitude for "{}"'.format(address))
        continue
    latitude = location.latitude
    longitude = location.longitude
    neighborhoods = neighborhoods.append({'District': nt.district,
                                          'Neighborhood': nt.neighbourhood,
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)

cannot find latitude and longitude for "East Bayfront, Old Toronto, Toronto, Ontario"
cannot find latitude and longitude for "Ordinance Triangle, Old Toronto, Toronto, Ontario"
cannot find latitude and longitude for "Regal Heights, Old Toronto, Toronto, Ontario"
cannot find latitude and longitude for "Clanton Park, North York, Toronto, Ontario"
cannot find latitude and longitude for "Humbermede, North York, Toronto, Ontario"
cannot find latitude and longitude for "Ledbury Park, North York, Toronto, Ontario"
cannot find latitude and longitude for "Pelmo Park-Humberlea, North York, Toronto, Ontario"
cannot find latitude and longitude for "Westminster–Branson, North York, Toronto, Ontario"
cannot find latitude and longitude for "Birch Cliff Heights, Scarborough, Toronto, Ontario"
cannot find latitude and longitude for "Tam O'Shanter-Sullivan, Scarborough, Toronto, Ontario"
cannot find latitude and longitude for "Briar Hill–Belgravia, York, Toronto, Ontario"
cannot find latitude and longit

Quickly examine the resulting dataframe.


In [35]:
neighborhoods.head()

,District,Neighborhood,Latitude,Longitude
0,Old Toronto,Alexandra Park,43.650787,-79.404318
1,Old Toronto,The Annex,43.670338,-79.407117
2,Old Toronto,Baldwin Village,43.677689,-79.408164
3,Old Toronto,Cabbagetown,43.664473,-79.366986
4,Old Toronto,CityPlace,43.639248,-79.396387


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.


In [36]:
print('The dataframe has {} neighborhoods.'.format(
        neighborhoods.shape[0]
    )
)

The dataframe has 204 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [37]:
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [38]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [39]:
CLIENT_ID = "5ZYZWNP1XIE3MUZKFCASPSJIOR13DKAS3KVMOGZXH4WQ15R3" # your Foursquare ID
CLIENT_SECRET = "D4K1ZKCUYZBKUEQOJWAHDGRHHMI3ZPZFFE0XHCSBFZKYOVMD" # your Foursquare Secret
ACCESS_TOKEN = "DABR2LI51C32L4KAC2BZYTG43U4TEYOYS2EBYQQAEOOB5TTD" # your FourSquare Access Token
VERSION = '20210223' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ZYZWNP1XIE3MUZKFCASPSJIOR13DKAS3KVMOGZXH4WQ15R3
CLIENT_SECRET:D4K1ZKCUYZBKUEQOJWAHDGRHHMI3ZPZFFE0XHCSBFZKYOVMD


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


In [40]:
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_name

'Alexandra Park'

Get the neighborhood's latitude and longitude values.


In [41]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Alexandra Park are 43.650786999999994, -79.40431814731767.


#### Now, let's get the top 100 venues that are in Agincourt within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [42]:
url = 'https://api.foursquare.com/v2/venues/search'
params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v=VERSION,
ll='{},{}'.format(neighborhood_latitude, neighborhood_longitude),
oauth_token=ACCESS_TOKEN,
radius=500,
limit=LIMIT
)
params

{'client_id': '5ZYZWNP1XIE3MUZKFCASPSJIOR13DKAS3KVMOGZXH4WQ15R3',
 'client_secret': 'D4K1ZKCUYZBKUEQOJWAHDGRHHMI3ZPZFFE0XHCSBFZKYOVMD',
 'v': '20210223',
 'll': '43.650786999999994,-79.40431814731767',
 'oauth_token': 'DABR2LI51C32L4KAC2BZYTG43U4TEYOYS2EBYQQAEOOB5TTD',
 'radius': 500,
 'limit': 100}

Send the GET request and examine the resutls


In [43]:
results = requests.get(url=url, params=params).json()
results['response']['venues'][:5]

[{'id': '4c30d270452620a196b01f0f',
  'name': 'Alexandra Park',
  'location': {'crossStreet': 'btwn Dundas, Carr & Denison',
   'lat': 43.650582680667924,
   'lng': -79.40453778486872,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.650582680667924,
     'lng': -79.40453778486872}],
   'distance': 28,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['275 Bathurst St. (btwn Dundas, Carr & Denison)',
    'Toronto ON',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d163941735',
    'name': 'Park',
    'pluralName': 'Parks',
    'shortName': 'Park',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1614371812',
  'hasPerk': False},
 {'id': '4c1a514db4e62d7f1697d793',
  'name': 'Bathurst Skatepark',
  'location': {'address': 'Bathurst',
   'lat': 43.650943,
   'lng': -79.404699,
   'labeledLatLngs': [{'label': 'display'

From the Foursquare lab in the previous module, we know that all the information is in the _response.venues_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [44]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [45]:
# venues = results['response']['groups'][0]['items']
venues = results['response']['venues']  
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,categories
0,Alexandra Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.650583,-79.404538,Park
1,Bathurst Skatepark,"[{'id': '4bf58dd8d48988d167941735', 'name': 'S...",43.650943,-79.404699,Skate Park
2,Alexandra Park/Dunbat Rink,"[{'id': '4bf58dd8d48988d168941735', 'name': 'S...",43.651383,-79.404582,Skating Rink
3,Bathurst Ice rink,"[{'id': '4bf58dd8d48988d168941735', 'name': 'S...",43.650972,-79.404983,Skating Rink
4,Camp Splish Splash,"[{'id': '4bf58dd8d48988d162941735', 'name': 'O...",43.650687,-79.404266,Other Great Outdoors


And how many venues were returned by Foursquare?


In [46]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore Neighborhoods in Manhattan


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [77]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):  
    venue_lists = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('Getting nearby venues for neighborhood {}...'.format(name))
            
        """ create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        """
        url = 'https://api.foursquare.com/v2/venues/explore'
        params = dict(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            v=VERSION,
            ll='{},{}'.format(lat, lng),
            radius=radius,
            limit=LIMIT,
        )
            
        # make the GET request
        results = requests.get(url=url, params=params).json()
        lst = get_venue_list(results, name, lat, lng)
        if len(lst) > 20:
            venue_lists.append(lst)

    nearby_venues = pd.DataFrame([item for venue_list in venue_lists for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

def get_venue_list(results, name, lat, lng):
    response = results["response"]
    if 'venues' in response:
        venues = response['venues']
        # return only relevant information for each nearby venue
        return [(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            get_category_type(v)) for v in venues]
    else:
        venues = response['groups'][0]['items']
        # return only relevant information for each nearby venue
        return [(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues]

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.


In [78]:
# type your answer here
filtered_columns = ['Neighborhood', 'Latitude', 'Longitude']
names, latitudes, longitudes = neighborhoods.loc[:, filtered_columns].T.values
toronto_venues = getNearbyVenues(names, latitudes, longitudes)

Getting nearby venues for neighborhood Alexandra Park...
Getting nearby venues for neighborhood The Annex...
Getting nearby venues for neighborhood Baldwin Village...
Getting nearby venues for neighborhood Cabbagetown...
Getting nearby venues for neighborhood CityPlace...
Getting nearby venues for neighborhood Chinatown...
Getting nearby venues for neighborhood Church and Wellesley...
Getting nearby venues for neighborhood Corktown...
Getting nearby venues for neighborhood Discovery District...
Getting nearby venues for neighborhood Distillery District...
Getting nearby venues for neighborhood Entertainment District...
Getting nearby venues for neighborhood Fashion District...
Getting nearby venues for neighborhood Financial District...
Getting nearby venues for neighborhood Garden District...
Getting nearby venues for neighborhood Grange Park...
Getting nearby venues for neighborhood Harbord Village...
Getting nearby venues for neighborhood Harbourfront...
Getting nearby venues for ne

#### Let's check the size of the resulting dataframe


In [79]:
print(toronto_venues.shape)
toronto_venues.head()

(4450, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alexandra Park,43.650787,-79.404318,Kanto,43.652167,-79.404843,Filipino Restaurant
1,Alexandra Park,43.650787,-79.404318,Market 707,43.652128,-79.404844,Food Court
2,Alexandra Park,43.650787,-79.404318,Buddha's Vegetarian,43.651904,-79.403312,Vegetarian / Vegan Restaurant
3,Alexandra Park,43.650787,-79.404318,#Hashtag Gallery,43.651830,-79.408103,Art Gallery
4,Alexandra Park,43.650787,-79.404318,Montauk,43.652084,-79.406898,Bar


Let's check how many venues were returned for each neighborhood


In [80]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alexandra Park,100,100,100,100,100,100
Baldwin Village,28,28,28,28,28,28
Beaconsfield Village,83,83,83,83,83,83
Bloor West Village,40,40,40,40,40,40
Cabbagetown,47,47,47,47,47,47
Casa Loma,21,21,21,21,21,21
Chinatown,64,64,64,64,64,64
Church and Wellesley,76,76,76,76,76,76
CityPlace,42,42,42,42,42,42


#### Let's find out how many unique categories can be curated from all the returned venues


In [81]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 315 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [82]:
# one hot encoding
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stat

And let's examine the new dataframe size.


In [83]:
onehot.shape

(4450, 315)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [84]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stat

#### Let's confirm the new size


In [85]:
grouped.shape

(87, 315)

#### Let's print each neighborhood along with the top 5 most common venues


In [86]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alexandra Park----
                    venue  freq
0                     Bar  0.10
1                    Café  0.05
2    Caribbean Restaurant  0.05
3  Furniture / Home Store  0.05
4             Coffee Shop  0.03


----Baldwin Village----
            venue  freq
0     Coffee Shop  0.11
1            Café  0.11
2  Sandwich Place  0.11
3  History Museum  0.07
4    Burger Joint  0.04


----Beaconsfield Village----
         venue  freq
0          Bar  0.14
1   Restaurant  0.06
2         Café  0.05
3  Coffee Shop  0.04
4    Nightclub  0.04


----Bloor West Village----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.08
2  Pizza Place  0.05
3          Pub  0.05
4        Diner  0.05


----Cabbagetown----
               venue  freq
0  Indian Restaurant  0.06
1               Café  0.06
2         Restaurant  0.06
3        Coffee Shop  0.06
4          Gastropub  0.04


----Casa Loma----
            venue  freq
0  Sandwich Place  0.14
1  History Museum  0.10
2            Café  0.10
3  

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexandra Park,Bar,Café,Caribbean Restaurant,Furniture / Home Store,Coffee Shop,Bakery,Arts & Crafts Store,Gym / Fitness Center,Liquor Store,Dessert Shop
1,Baldwin Village,Coffee Shop,Café,Sandwich Place,History Museum,Burger Joint,Modern European Restaurant,Garden Center,Middle Eastern Restaurant,Steakhouse,Castle
2,Beaconsfield Village,Bar,Restaurant,Café,Coffee Shop,Nightclub,Vegetarian / Vegan Restaurant,Lounge,Theater,Bakery,Art Gallery
3,Bloor West Village,Coffee Shop,Café,Pizza Place,Pub,Diner,Sushi Restaurant,Yoga Studio,Burrito Place,Smoothie Shop,Sandwich Place
4,Cabbagetown,Indian Restaurant,Café,Restaurant,Coffee Shop,Gastropub,Italian Restaurant,Japanese Restaurant,Diner,Pizza Place,Bakery


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [89]:
grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stat

In [90]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 2, 3, 3, 4, 2, 1, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [91]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = neighborhoods

# merge grouped with neighborhoods to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

merged.head() # check the last columns!

,District,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Old Toronto,Alexandra Park,43.650787,-79.404318,2,Bar,Café,Caribbean Restaurant,Furniture / Home Store,Coffee Shop,Bakery,Arts & Crafts Store,Gym / Fitness Center,Liquor Store,Dessert Shop
1,Old Toronto,The Annex,43.670338,-79.407117,3,Pizza Place,Park,Bistro,Indian Restaurant,Thai Restaurant,Burrito Place,Middle Eastern Restaurant,Metro Station,Greek Restaurant,Bookstore
2,Old Toronto,Baldwin Village,43.677689,-79.408164,4,Coffee Shop,Café,Sandwich Place,History Museum,Burger Joint,Modern European Restaurant,Garden Center,Middle Eastern Restaurant,Steakhouse,Castle
3,Old Toronto,Cabbagetown,43.664473,-79.366986,3,Indian Restaurant,Café,Restaurant,Coffee Shop,Gastropub,Italian Restaurant,Japanese Restaurant,Diner,Pizza Place,Bakery
4,Old Toronto,CityPlace,43.639248,-79.396387,3,Coffee Shop,Gym,Café,Grocery Store,Park,Japanese Restaurant,Ice Cream Shop,Historic Site,Greek Restaurant,Sushi Restaurant


Finally, let's visualize the resulting clusters


In [92]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 2


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 3


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 4


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 5


In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) and [Polong Lin](https://www.linkedin.com/in/polonglin?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!


This notebook is part of a course on **Coursera** called _Applied Data Science Capstone_. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description         |
| ----------------- | ------- | ------------- | -------------------------- |
| 2020-11-26        | 2.0     | Lakshmi Holla | Updated the markdown cells |
|                   |         |               |                            |
|                   |         |               |                            |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
